In [1]:
!pip install --upgrade geopandas mapclassify folium

In [2]:
!pip install arcgis --no-deps # --use-deprecated=legacy-resolver # install ArcGIS library for geocoding
!pip install ujson requests_toolbelt requests_ntlm geomet
!pip install OSMPythonTools
!pip install cachetools requests_oauthlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 MB 18.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for arcgis: filename=arcgis-2.2.0.2-cp311-cp311-linux_x86_64.whl size=8487781 sha256=b5af0da3e02cb78e6fedd0d74594a15cbfc37a0dfa4e7659536d8c7a803fd78b
  Stored in directory: /home/jovyan/.cache/pip/wheels/93/b2/4b/2858c95895c2205a0bf9636081a49c7762f0d08559e37b23f8
Successfully built arcgis
  Obtaining dependency information for ujson from https://files.pythonhosted.org/packages/d3/93/de6fff3ae06351f3b1c372f675fe69bc180f93d237c9e496c05802173dd6/ujson-5.9.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached ujson-5.9.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.7 kB)
  Obtaining dependency information for geomet from https://files.pythonhosted.org/packages/af/90/3bc780df088d439714af8295196a4332a26559ae66fd99865e36f92efa9e/geomet-1.1.0-py3-none-any.whl.metadata
  Using cached geomet-1.

In [3]:
import geopandas as gpd
import arcgis # see if the install worked
import pandas as pd # us this to save the data as a CSV file on your Google Drive
import time
import mapclassify
import folium
import re # us this for regular expressions
import glob
import csv # use this to write reviews as CSV later

from OSMPythonTools.nominatim import Nominatim
from arcgis.gis import GIS # import the things we need
import sys
from arcgis.geocoding import geocode
from arcgis.geocoding import analyze_geocode_input

import requests
import logging

logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)
# create console handler
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

arcgis.geocoding.get_geocoders(GIS())

Connecting to portal: www.arcgis.com
Starting new HTTPS connection (1): www.arcgis.com:443
https://www.arcgis.com:443 "GET /sharing/info?f=json HTTP/1.1" 200 84
https://www.arcgis.com:443 "GET /sharing/rest/info?f=json HTTP/1.1" 200 None
https://www.arcgis.com:443 "GET /sharing/sharing/rest/info?f=json HTTP/1.1" 200 84
https://www.arcgis.com:443 "GET /sharing/rest/services?f=json HTTP/1.1" 200 84
Starting new HTTPS connection (1): www.arcgis.com:443
https://www.arcgis.com:443 "POST /sharing/rest/portals/self HTTP/1.1" 200 None
https://www.arcgis.com:443 "POST /sharing/rest/ HTTP/1.1" 200 None
https://www.arcgis.com:443 "POST /sharing/rest/portals/self/servers/computePrivateServiceUrl HTTP/1.1" 200 None


[<Geocoder url:"https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer">]

In [21]:
## Original dataset uploaded in Jupyter Notebook
import pandas as pd
Childcare_Centers = pd.read_csv('Data/Licensed_Childcare_Centers.csv')
Childcare_Centers 

,COUNTY,CENTER,PHONE,AGES,CAPACITY,CITY,ADDR1,ADDR2,ZIP,INSPECTIONS
0,Passaic,El Mundo De Colore/World of Colors,973-523-0919,2½ to 6,154,Paterson,44 Ward Street,NaN,07501,Inspection Reports (http://www.nj.gov/dcf/fami...
1,Passaic,El Mundo Del Nino/A Child's World,973-279-7100,2½ to 6,116,Paterson,433 Main Street,NaN,07501,Inspection Reports (http://www.nj.gov/dcf/fami...
2,Hudson,Concordia Learning Center,201-876-5432,2½ to 6,120,Jersey City,761 Summit Ave.,NaN,07307,Inspection Reports (http://www.nj.gov/dcf/fami...
3,Mercer,The Children's Home Society of NJ Afterschool ...,609-695-6274,6 to 13,180,Trenton,1300 Stuyvesant Avenue,NaN,08618,Inspection Reports (http://www.nj.gov/dcf/fami...
4,Burlington,Willingboro BOE at Hawthorne School,609-835-8600,2½ to 13,60,Willingboro,84 Hampshire Lane,NaN,08046,Inspection Reports (http://www.nj.gov/dcf/fami...
...,...,...,...,...,...,...,...,...,...,...
4158,Passaic,Enchanted Learning Daycare and Preschool,862-248-0904,0 to 13,40,Pompton Lakes,235 Wanaque Avenue,NaN,07442,Inspection Reports (http://www.nj.gov/dcf/fami...
4159,Middlesex,STEM Academy for Young Kids,732-243-9683,0 to 13,189,Edison,1665 Oak Tree Road,Suite 400,08820,Inspection Reports (http://www.nj.gov/dcf/fami...
4160,Middlesex,Montessori Matters Preschool Inc.,732-317-8955,0 to 13,112,Piscatway,111 Suttons Lane,NaN,08854,Inspection Reports (http://www.nj.gov/dcf/fami...
4161,Somerset,The Learning Experience,732-646-6540,0 to 13,163,Bridgewater,1940 Route 22,NaN,08805,Inspection Reports (http://www.nj.gov/dcf/fami...


In [22]:
# to create a new dataset with the first 250 rows
Childcare_Centers.drop(Childcare_Centers.index[251:4162],inplace=True)
Childcare_Centers.drop("INSPECTIONS", axis = 1, inplace=True)   
Childcare_Centers
Childcare_Centers.to_csv('Childcare_Centers_New.csv')

In [23]:
# to run the dataset through the ESRI geocoder 

def geocode_Childcare_Centers_NJ(inputfilepath, outputfilepath): # function for geocoding all reviews from our scraped files
  
  with open(inputfilepath, 'r') as read_obj: # first load the file
      dict_reader = csv.DictReader(read_obj) # pass the file object to DictReader() to get the DictReader object
      input_rows = list(dict_reader) # get a list of dictionaries from dct_reader

  input_rows_geocoded = [] # initialize an empty list to hold geocoded reviews

  for centers in input_rows: # iterate through list
    if len(centers['ZIP']) < 5:
      centers['ZIP'] = centers['ZIP'].rjust(5, '0')
      print("I added a leading 0 to the ZIP!")
    multi_field_address = { 
                        "Address" : centers['ADDR1'],
                        "City" : centers['CITY'],
                        "County" : centers['COUNTY'],
                        "Postal" : centers['ZIP']
                      }
    centers_geocoded = centers # duplicate review
    centers_geocoded['ESRI_Review_Needed'] = 1 # default to needing manual review unless we get a point or subaddress back
    if len(geocode(multi_field_address)) > 0: # if you get any results
      geocodeoutput = geocode(multi_field_address)[0] # geocodes the childcare centers location and gives first choice for now  
      print("Geocoded " + multi_field_address['Address'] + ", " + multi_field_address['City'] + ", " + multi_field_address['County'] + ", "+ multi_field_address['Postal']) # print name of place geocoded
      centers_geocoded['ESRI_Lon'] = geocodeoutput['attributes']['DisplayX'] # saves Lon
      centers_geocoded['ESRI_Lat'] = geocodeoutput['attributes']['DisplayY'] # saves Lat
      centers_geocoded['ESRI_Match_Score'] = geocodeoutput['score'] # saves score
      centers_geocoded['ESRI_Match_Type'] = geocodeoutput['attributes']['Addr_type'] # saves address type
      if geocodeoutput['attributes']['Addr_type'] == "PointAddress" or geocodeoutput['attributes']['Addr_type'] == "Subaddress":
        centers_geocoded['ESRI_Review_Needed'] = 0 # remove review flag if one of these match types is received
      centers_geocoded['ESRI_Match_Address'] = geocodeoutput['attributes']['LongLabel'] # saves match address
    else: # if you get no results (blanks in all input fields, typically)
        centers_geocoded['ESRI_Lon'], centers_geocoded['ESRI_Lat'], centers_geocoded['Match_Score'], centers_geocoded['Match_Type'], centers_geocoded['Match_Address'] = "", "", 0, "", ""
    input_rows_geocoded.append(centers_geocoded) # adds to new dictionary

  keys = input_rows_geocoded[0].keys() # get the list of columns to write

  with open(outputfilepath, 'w')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(input_rows_geocoded)
  print("Wrote " + outputfilepath + "!")

In [24]:
geocoding_input_filepath = "Childcare_Centers_New.csv"
geocoding_output_filepath = "Childcare_Centers_New_Geocoded_ESRI.csv"

geocode_Childcare_Centers_NJ(geocoding_input_filepath, geocoding_output_filepath)

https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 608
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 608
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 609
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 609
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 612
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 612
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 616


Geocoded 44 Ward Street, Paterson, Passaic, 07501
Geocoded 433 Main Street, Paterson, Passaic, 07501
Geocoded 761 Summit Ave., Jersey City, Hudson, 07307


https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 616
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 620
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 620
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 641
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 641
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 932


Geocoded 1300 Stuyvesant Avenue, Trenton, Mercer, 08618
Geocoded 84 Hampshire Lane, Willingboro, Burlington, 08046
Geocoded 50 Rev. Dr. Martin Luther King, Willingboro, Burlington, 08046


https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 932
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 616
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 616
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 619
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 619


...

Geocoded 2562 Tilton Rd, Egg Harbor Township, Atlantic, 08234
Geocoded 1076 Wayside Rd, Ocean Township, Monmouth, 07712
Geocoded 1600 Stuyvesant Ave, Trenton, Mercer, 08618


https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 970
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 617
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 617
https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 616


Geocoded 540 Route 57e, Port Murray, Warren, 07865
Geocoded 186-190 Butler Street, Paterson, Passaic, 07524


https://geocode.arcgis.com:443 "POST /arcgis/rest/services/World/GeocodeServer/findAddressCandidates HTTP/1.1" 200 616


Geocoded 10 Grove Street, Cherry Hill, Camden, 08002
Wrote Childcare_Centers_New_Geocoded_ESRI.csv!
